In [595]:
"""Exploring BeautifulSoup to scrape syosetu page"""
from bs4 import BeautifulSoup
from urllib.request import urlopen
import matplotlib.pyplot as plt 
from datetime import datetime 
from typing import List, Tuple
import pandas as pd 
import string, re

url = r"https://ncode.syosetu.com/n7855ck/"
url = r"https://ncode.syosetu.com/n9669bk/"
html = urlopen(url)
bs = BeautifulSoup(html, 'lxml')

NaT


In [596]:
def remove_punctuation(data: List[object], join=False) -> List[str]:
    """
    Remove punctuation from each string in a list of `bs4.element.Tag`s
    """
    if isinstance(data, str):
        return re.sub(r'([^\w\s]|_)','', data)
    
    print(data)
    
    for i, t in enumerate(data):
        data[i] = re.sub(r'([^\w\s]|_)','', t.string)
    
    if join: 
        return "".join(data)
    
    return data 

In [597]:
def get_novel_info(bs: BeautifulSoup) -> List[str]:
    title = bs.find(name='title').string
    
    author = bs.find(
        'div', class_='novel_writername'
    ).contents[1].string
    
    return title, author

First line(s) of summary are often some sort of advertisement. Ignore this for now.

In [598]:
def get_summary(bs: BeautifulSoup) -> str:
    summary = bs.find('div', id='novel_ex').stripped_strings
    return ''.join(list(summary))

In [599]:
def get_volume_titles(bs: BeautifulSoup) -> List[str]:
    volumes = bs.find_all('div', class_='chapter')
    
    if not volumes: return []
    
    for i, t in enumerate(volumes):
        volumes[i] = re.sub(r'([^\w\s]|_)','', t.string)
        
    return volumes

In [600]:
def get_section_titles(bs: BeautifulSoup) -> List[str]:
    chapters = bs.find_all('div', class_='chapter')
    return remove_punctuation(chapters)

In [601]:
# kanji to number mapping 
sinogram_mapping = {'〇': 0, '一': 1, '二': 2,
                    '三': 3, '四': 4, '五': 5,
                    '六': 6, '七': 7, '八': 8,
                    '九': 9, '十': 10, '百': 100,
                    '千': 1000, '万': 10000}

def remove_non_cn_num(cn: str) -> str:
    return re.sub("[^〇一二三四五六七八九十百千万]", "", cn)

def cn_to_arb_number(num: str, mapping=sinogram_mapping) -> int:
    # https://stackoverflow.com/questions/15076443/convert-numbers-in-chinese-characters-to-arabic-numbers
        
    num = remove_non_cn_num(num)
    length = len(num)
    
    if length == 0:
        return None 
    elif 0 < length < 2:
        return mapping[num]
    elif length < 3:
        ones = mapping[num[1]]
        if ones < 10: 
            return mapping[num[0]] + ones 
        else: 
            return mapping[num[0]] * ones
    
    X = mapping[num[-1]]
    for i in range(0, length-1, 2):
        X += mapping[num[i]] * mapping[num[i+1]]
    
    return X

def get_chapter_link_and_title(subtitle: object) -> Tuple[int, str, str]:
    """Extract chapter index, link, and title"""
    
    link = "https://ncode.syosetu.com/" + subtitle.a.attrs['href']
    title = subtitle.a.string
    
    title = re.split("\s", title)
    
    if len(title) < 2:
        print(f"{title} could not be parsed into (index, title)")
        return None, link, title[0]
    
    elif len(title) > 2:
        index = title[0]
        title = ''.join(title)
        
    else:
        index, title = title 
    
    try:
        index = int(index)
    except ValueError:
        index = cn_to_arb_number(index)
    
    title = remove_punctuation(title)
    return index, link, title 

# ----------------------------------- tests ---------------------------------- #

def _test_cn_to_arb_number():
    test_data = ['一百一十四', ' 四十五', '十五', '十']
    true_values = [114, 45, 15, 10]
    
    for arb, cn in zip(true_values, test_data):
        assert arb == cn_to_arb_number(cn)
    
    return True


def _test_get_chapter_link_and_title():
    
    test_data = [
        """<dd class="subtitle">
        <a href="/n7855ck/1/">第一話　運命の出会い</a>
        </dd>
        """,
        """<dd class="subtitle">
        <a href="/n7855ck/51/">第五十一話　リツハルドの孤独な十年間　前編</a>
        </dd>
        """,
        """<dd class="subtitle">
        <a href="/n5943db/3/">2 ひとりぼっちに</a>
        </dd>
        """
    ]
    
    true_output = [
        (1, 'https://ncode.syosetu.com//n7855ck/1/', '運命の出会い'), 
        (51, 'https://ncode.syosetu.com//n7855ck/51/', '第五十一話リツハルドの孤独な十年間前編'),
        (2, 'https://ncode.syosetu.com//n5943db/3/', 'ひとりぼっちに')
    ]
    
    for true_val, test_val in zip(true_output, test_data):
        test_val = BeautifulSoup(test_val).find('dd')
        print(get_chapter_link_and_title(test_val))
        assert true_val == get_chapter_link_and_title(test_val)
        
    return True


In [602]:
def strip_dt(timestamp: str) -> str:
    """Strip non-datetime characters timestamp"""
    return re.sub("[^\d/:\s]", '', timestamp).strip()

def to_dt(timestamp: str) -> datetime:
    """Parse timestamp string as datetime object"""
    return datetime.strptime(timestamp, r"%Y/%m/%d %H:%M")   

def get_chapter_times(chapter: object) -> Tuple[datetime, datetime]:
    """Get time when chapter was created and when it was last edited, if available"""
    chapter_time = chapter.dt 
    creation_time = to_dt(strip_dt( chapter_time.contents[0] ))
    
    if chapter_time.span is not None:
        edit_time = to_dt(strip_dt( chapter_time.span.attrs['title'] ))
        return creation_time, edit_time
    else:
        return creation_time, None
    
# ----------------------------------- tests ---------------------------------- #

def _test_strip_dt():
    testset = [
        "2014/12/22 22:01 改稿",
        "!@(*&!@<><><> 2014/12/22 22:01 &^%!@(*&",
        "2014/12/22 22:01"
    ]
    
    true_value = "2014/12/22 22:01"
    for test in testset:
        assert true_value == strip_dt(test)
    
    return True 

def _test_get_chapter_info():

    test_data = [
        # test with edit time 
        """<dl class="novel_sublist2">
        <dd class="subtitle">
        <a href="/n7855ck/9/">第九話　雪国生活一日目</a>
        </dd>
        <dt class="long_update">
        2014/12/21 20:30<span title="2021/07/07 22:11 改稿">（<u>改</u>）</span></dt>
        </dl>""",
        
        # test without edit time 
        """<dl class="novel_sublist2">
        <dd class="subtitle">
        <a href="/n7855ck/23/">第二十三話　近づいた距離</a>
        </dd>
        <dt class="long_update">
        2015/01/04 00:00</dt>
        </dl>
        """
    ]
    
    true_output = [
        (datetime(2014, 12, 21, 20, 30), datetime(2021, 7, 7, 22, 11)),
        (datetime(2015, 1, 4, 0, 0), None)
    ]
    
    for data, true_output in zip(test_data, true_output):
        data = BeautifulSoup(data)
        test_output = get_chapter_times(data)
        print(test_output)
        try:
            assert true_output == test_output
        except AssertionError:
            raise AssertionError(f"True output: {true_output}\nTest output: {test_output}")            

    return True 

In [603]:
def get_chapter_info(bs: BeautifulSoup) -> pd.DataFrame:
    chapter_data = dict(
        index= [], link = [], title = [], creation_time = [], edit_time = []
    )
    
    for chapter in bs.find_all('dl', class_='novel_sublist2'):
        index, link, title = get_chapter_link_and_title(chapter)
        creation_time, edit_time = get_chapter_times(chapter)
        
        chapter_data['index'].append(index)
        chapter_data['link'].append(link)
        chapter_data['title'].append(title)
        chapter_data['edit_time'].append(edit_time)
        chapter_data['creation_time'].append(creation_time)
    
    df = pd.DataFrame(chapter_data)
    return df 

In [604]:
df = get_chapter_info(bs)

['プロローグ'] could not be parsed into (index, title)
['第一話「もしかして：異世界」'] could not be parsed into (index, title)
['第二話「ドン引きのメイドさん」'] could not be parsed into (index, title)
['第三話「魔術教本」'] could not be parsed into (index, title)
['第四話「師匠」'] could not be parsed into (index, title)
['第五話「剣術と魔術」'] could not be parsed into (index, title)
['第六話「尊敬の理由」'] could not be parsed into (index, title)
['第七話「友達」'] could not be parsed into (index, title)
['第八話「鈍感」'] could not be parsed into (index, title)
['第九話「緊急家族会議」'] could not be parsed into (index, title)
['第十話「伸び悩み」'] could not be parsed into (index, title)
['第十一話「離別」'] could not be parsed into (index, title)
['第十二話「お嬢様の暴力」'] could not be parsed into (index, title)
['第十三話「自作自演？」'] could not be parsed into (index, title)
['第十四話「凶暴性、いまだ衰えず」'] could not be parsed into (index, title)
['第十五話「職員会議と日曜日」'] could not be parsed into (index, title)
['第十六話「お嬢様は十歳」'] could not be parsed into (index, title)
['第十七話「言語学習」'] could not be parsed into (index, title)
['第

In [641]:
url = r"https://ncode.syosetu.com/n7855ck/51/"
html = urlopen(url)
bs = BeautifulSoup(html, 'lxml')

In [674]:
body = ""
for line in bs.find('div', id='novel_honbun', class_='novel_view').children:
    try:
        if line.br: continue
        body += remove_punctuation(line.string.strip())
    except: continue


'祖父が死んだ太陽が沈まない森が一番輝くという祖父の好きだった季節にはかなくなった祖父は自分に伝統工芸や猟の仕方獲物の解体について領主としての仕事全てを授けたので悔いは無いと言っていた数日後に眠るようにしてこの世を去っていく本当に長い間頑張って来たと思うどうか安らかに眠ってくれと祖母の隣に弔ったそれからは怒涛の毎日であった祖父が倒れてから領主代理をしていたとはいえいきなり完璧な仕事が出来る訳もないわたわたと数ヶ月間慌しく仕事に追われていると両親に話があると呼び出される二人揃って嫌な予感しかしないという推測は見事に的中をしてしまった父は言うちょっと今から寒くなるので暖かい場所を目指して冒険に行って来るねとそれを聞いて別に驚きもしなかった父はずっと世界の探険に出る事を長年望んでいたからだそんな風に家族を置いていつ帰ってくるかも分からないという冒険の旅を許さなかったのは祖父で父親はやっと解放されたということになるだが想定外だったのはその後に続く母親の言葉だったお父さんが心配だからお母さんもついて行くねえといやいや二人揃って破壊力も二倍だろう両親は何と表現すればいいのかふわふわしているというか浮世離れをしているというかでもまあ両親は堅苦しいこの村に居るよりはのびのびと出来る環境の中に生きる方が合っているのかもしれないと思い旅立ちを止める事はしなかった父は旅立ちの準備にはじっくりと時間を掛けていたその間に母は通いのお手伝いさんを探したりなどの様々な手配をしてくれるそして出発の朝を迎えたリッちゃんごめんねえ大変なときにこんな事になってえ大丈夫母さん達には何も期待していなかったから割と失礼なことを言っても父は本当\u3000よかったあとのんびりした様子で呟く母も暢気にニコニコと微笑むだけだったあらまあお父さんリッちゃん見てえ綺麗な蝶さんえうわっ空をふわふわと漂うようにしている蝶を見て父は驚きの声を上げるああれは世界的に珍しいとされている伝説の蝶ヘレナモルフォ\u3000どうしてこんな所にそんな風に早口で捲くし立てながら父は蝶を追って走り出すあらあら大変母はこちらにひらひらと手を振ってからそれ走っているの\u3000と疑ってしまうような遅さでさかさかと父の後を追って行ったなんというか脱力心配でしかない両親は特別な前触れも無く旅立ってしまう二人を安心させる為に領主としての意気込みとか